In [18]:
import pandas as pd
import numpy as np
import finpy_tse as fpy
import datetime
import jdatetime as jdt

from CONFIG import DB_PASS as PASS
from metafid.mfw import DB


In [2]:
db = DB(dbname="metafid", user="postgres", pass_=PASS)

### Stock List

In [42]:
tickers = db.query_all(table="tsedata_ticker", cols="(symbol, symbol_far,market_id)")
tse_tickers= tickers[tickers.market_id=="1"]

### Update `tsedata_section` DB

In [52]:
def update_ri(stock_list):

    # get max date
    date_id = db.query_all(table="tsedata_section", cols="(date_id)")
    max_date = date_id.date_id.max()
    # today jdate
    today = jdt.date.today().strftime("%Y-%m-%d")
    # create start_date
    def start_date(date):
        if len(date_id.date_id):
            date = date + datetime.timedelta(days=1)
            return jdt.date.fromgregorian(year = date.year, day=date.day, month=date.month).strftime("%Y-%m-%d")
        return "1400-01-01"
    # take data
    if start_date(max_date) != today:
        df = pd.DataFrame()
        for stock in stock_list.symbol_far:
            df_ri = fpy.Get_RI_History(
                stock=stock,
                start_date=start_date(max_date),
                end_date=today,
                ignore_date=False,
                show_weekday=False,
                double_date=True).rename(columns=str.lower).drop(["name","market"], axis=1)
            df= pd.concat([df, df_ri])
        df = df.merge(stock_list, left_on="ticker", right_on="symbol_far", how="inner")
        df.drop(["ticker","symbol_far", "market_id" ], axis=1, inplace=True)
        df.rename(columns={"date":"date_id", "symbol":"symbol_id"}, inplace=True)
        return db.insert_data(table="tsedata_section", df=df)
    return print("DB is update")
update_ri(tse_tickers)

### Update `tsedata_histprice` DB

In [60]:
def update_hist_price(stock_list):

    # get max date
    date_id = db.query_all(table="tsedata_histprice", cols="(date_id)")
    max_date = date_id.date_id.max()
    # today jdate
    today = jdt.date.today().strftime("%Y-%m-%d")
    # create start_date
    def start_date(date):
        if len(date_id.date_id):
            date = date + datetime.timedelta(days=1)
            return jdt.date.fromgregorian(year = date.year, day=date.day, month=date.month).strftime("%Y-%m-%d")
        return "1400-01-01"
    # take data and update DB
    if start_date(max_date) != today:
        df = pd.DataFrame()
        for stock in stock_list.symbol_far:
            df_hist = fpy.Get_Price_History(
                stock=stock,
                start_date=start_date(max_date),
                end_date=today,
                ignore_date=False,
                adjust_price=True,
                show_weekday=False,
                double_date=True).rename(columns=str.lower).drop(["open","high","low", "close", "final", "no","name","market"], axis=1)
            df_hist.columns = [col.replace("adj ", "") for col in df_hist.columns]
            df= pd.concat([df, df_hist])
            
        df = df.merge(stock_list, left_on="ticker", right_on="symbol_far", how="inner")
        df.drop(["ticker","symbol_far", "market_id" ], axis=1, inplace=True)
        df.rename(columns={"date":"date_id", "symbol":"symbol_id"}, inplace=True)
        return db.insert_data(table="tsedata_histprice", df=df)
    return print("DB is update")
update_hist_price(tse_tickers)